# Troubleshooting missing String data


[It has been identified](https://github.com/opentargets/platform/issues/1330) that `experiments` fields are missing from the GraphQL data. 

* String API [response](https://string-db.org/cgi/network?taskId=b6xg4PlcNVGm)
* OT GraphQL [response](https://api-beta-dot-open-targets-eu-dev.appspot.com/api/v4/graphql/browser?query=%7B%0A%20%20target(ensemblId%3A%20%22ENSG00000111640%22)%20%7B%0A%20%20%20%20approvedSymbol%0A%20%20%20%20interactions(sourceDatabase%3A%20%22string%22)%20%7B%0A%20%20%20%20%20%20count%0A%20%20%20%20%20%20rows%20%7B%0A%20%20%20%20%20%20%20%20intA%0A%20%20%20%20%20%20%20%20intB%0A%20%20%20%20%20%20%20%20targetA%20%7B%0A%20%20%20%20%20%20%20%20%20%20id%0A%20%20%20%20%20%20%20%20%20%20approvedSymbol%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20targetB%20%7B%0A%20%20%20%20%20%20%20%20%20%20id%0A%20%20%20%20%20%20%20%20%20%20approvedSymbol%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20evidences%20%7B%0A%20%20%20%20%20%20%20%20%20%20interactionDetectionMethodShortName%0A%20%20%20%20%20%20%20%20%20%20interactionDetectionMethodMiIdentifier%0A%20%20%20%20%20%20%20%20%20%20evidenceScore%0A%20%20%20%20%20%20%20%20%20%20interactionScore%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A)

What the hell???

1. Check the input file from [platform input support](https://github.com/opentargets/platform-input-support/blob/master/config.yaml)
2. Find [String DB](https://stringdb-static.org/download/protein.links.detailed.v11.0/9606.protein.links.detailed.v11.0.txt.gz) file.

In [4]:
%%bash

curl -s https://stringdb-static.org/download/protein.links.detailed.v11.0/9606.protein.links.detailed.v11.0.txt.gz | gzcat | head

protein1 protein2 neighborhood fusion cooccurence coexpression experimental database textmining combined_score
9606.ENSP00000000233 9606.ENSP00000272298 0 0 332 62 181 0 125 490
9606.ENSP00000000233 9606.ENSP00000253401 0 0 0 0 186 0 56 198
9606.ENSP00000000233 9606.ENSP00000401445 0 0 0 0 159 0 0 159
9606.ENSP00000000233 9606.ENSP00000418915 0 0 0 61 158 0 542 606
9606.ENSP00000000233 9606.ENSP00000327801 0 0 0 88 78 0 89 167
9606.ENSP00000000233 9606.ENSP00000466298 0 0 0 141 131 0 98 267
9606.ENSP00000000233 9606.ENSP00000232564 0 0 0 62 171 0 56 201
9606.ENSP00000000233 9606.ENSP00000393379 0 0 0 61 131 0 43 150
9606.ENSP00000000233 9606.ENSP00000371253 0 0 0 61 0 0 224 240


So the column is in the input file. What happens with it in the platform input support?

**Solution:** The column header is different from what the script is expecting... 

```python
    # The following STRING channels can be mapped to detection methods on MI onotology:
    detection_method_mapping = {
        'coexpression': {'name': 'coexpression', 'mi_id': 'MI:2231'},
        'coexpression_transferred': {'name': 'coexpression_transferred', 'mi_id': ''},
        'neighborhood': {'name': 'neighborhood', 'mi_id': 'MI:0057'},
        'neighborhood_transferred': {'name': 'neighborhood_transferred', 'mi_id': ''},
        'fusion': {'name': 'fusion', 'mi_id': 'MI:0036'},
        'homology': {'name': 'homology', 'mi_id': 'MI:2163'},
        'experiments': {'name': 'experiments', 'mi_id': 'MI:0591'},
        'experiments_transferred': {'name': 'experiments_transferred', 'mi_id': ''},
        'cooccurence': {'name': 'cooccurence', 'mi_id': 'MI:2231'},
        'database': {'name': 'database', 'mi_id': ''},
        'database_transferred': {'name': 'database_transferred', 'mi_id': ''},
        'textmining': {'name': 'textmining', 'mi_id': 'MI:0110'},
        'textmining_transferred': {'name': 'textmining_transferred', 'mi_id': ''},
    }
```

It says `experiment` instead of `experimental`. Holy shit.